# **Bank Customer Churn Prediction Using ANN:**

**Importing Required Libraries**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = [10, 5]

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/bank-customer-churn-modeling/Churn_Modelling.csv')
df.head()

****FirstofAll Lets remove some Unwanted columns from the dataset****

In [ ]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace=True)
df

In [ ]:
df.info()

In [ ]:
# Finding Missing Values

df.isnull().sum()

****We are checking how much a column has all the values. As this can be done with only numerical column so we ignore the categorical columns****

In [ ]:
columns = [col for col in df.columns if df[col].dtype != 'O']
df[columns].describe()

****Check How our Dataset looks? Is it Balanced or Imbalance?****

In [ ]:
sns.set_theme(style="darkgrid")
sns.countplot(df['Exited'], palette='Set1')
plt.xlabel('Customers Exited or Not', size = 15)
plt.show()

In [ ]:
customer_not_exited = df[df.Exited == 0]
customer_exited = df[df.Exited == 1]

In [ ]:
print(customer_not_exited.shape, customer_exited.shape)

****We can see a dataset Imbalanced, so need to use some technique to make it Balanced. However, we can do it later****

# **Data Visualisations**

In [ ]:
exited = df[df.Exited == 1].Tenure
not_exited = df[df.Exited == 0].Tenure

In [ ]:
plt.hist([exited, not_exited], label = ['Yes', 'No'])
plt.xlabel('Tenure', color = 'r')
plt.ylabel('Number of Customers', color = 'green')
plt.title('Customers Loyalty', color = 'magenta')
plt.legend()
plt.show()

****By looking this graph we can say that If customer stays mrore than 5 months they likely happy with the service of bank****

In [ ]:
sns.countplot(df['Geography'], data = df, palette='Paired')
plt.title('Bank-Customers from Each Country', size = 20, color = 'indigo')
plt.xlabel('Countries', size = 15, color = 'coral')
plt.ylabel('Number of Customers', size=15, color = 'peru')
plt.show()

In [ ]:
sns.countplot(df['Gender'],hue=df['Exited'], palette='husl')
plt.legend(labels=['Not_Exited', 'Exited'])
plt.title('Gender ratio in Churn-prediction', size = 20, color = 'crimson')
plt.xlabel('Gender', size = 15, color = 'sienna')
plt.ylabel('Number of Customers', size=15, color = 'salmon')
plt.show()

In [ ]:
# Check how many Customers are actived and still they want to move to another bank?

sns.countplot('IsActiveMember',hue = 'Exited', data=df, palette='hls')
plt.legend(labels=['Not_Exited', 'Exited'])
plt.title('Members Activity in Bank', size = 20, color = 'teal')
plt.xlabel('Active Members', size = 15, color = 'royalblue')
plt.ylabel('Number of Customers', size=15, color = 'orange')
plt.show()

****Will check how many columns are categorical and we will change into numerical data****

In [ ]:
for col in df.columns:
    if df[col].dtype == 'O':
        print(f'{col}: {df[col].unique()}')

****Unique values are less in a each categorical column so we won't use OHE or LabelEncoding.We can do it with replace function.****

In [ ]:
df['Geography'] = df['Geography'].replace({'France': 0, 'Spain': 1, 'Germany': 2})
df['Gender'] = df['Gender'].replace({'Female': 0, 'Male': 1})

In [ ]:
df.head()

****Now, We have all Numerical Data so,we can divide the dataset into X, Y. So, In X we take Independent Features and in Y we take our Target class, which is Exited****

In [ ]:
X = df.drop('Exited',axis=1)
Y = df['Exited']

In [ ]:
X.shape, Y.shape

****So,Here we perform Oversampling technique for making Balanced Dataset,which basically make more data within the existing data,for the lower data class so, that both target_class comes on a same page and we can make prediction better. Although, You can also use Undersampling technique but, in that the chances of data loss will be high, hence we used OverSampling here****

In [ ]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(random_state=42)
X,Y = smk.fit_resample(X, Y)

In [ ]:
X.shape, Y.shape

In [ ]:
sns.set_theme(style="darkgrid")
sns.countplot(Y, palette='Set1')
plt.xlabel('Customers Exited or Not', size = 15)
plt.show()

****Finally, we make our Dataset Balanced****

In [ ]:
X

****Here, we see many columns have various values, so in order to get good accuracy we need to do scaling on these columns so that every columns values falls between 0 to 1****

In [ ]:
cols_to_scale = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])

In [ ]:
X

# **Model Training Using ANN:**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(20, input_shape=(10,), activation='relu'),
    keras.layers.Dense(14, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',      # As,we have Binary Prediction, we used this loss func
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
Y_prediction = model.predict(X_test)
Y_prediction[:5]

****So, It gives prediction between 0 to 1 values. Let's take any value which is greater than 0.5,it would be 1 and any value falls less than 0.5, will put 0 instead.****

In [ ]:
Y_Predict_values = []

for ele in Y_prediction:
    if ele > 0.5:
        Y_Predict_values.append(1)
    else:
        Y_Predict_values.append(0)

In [ ]:
Y_Predict_values[:5]

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, Y_Predict_values))

In [ ]:
cm = tf.math.confusion_matrix(labels=y_test, predictions=Y_Predict_values)

plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

****We can see Model Predicts Good, although there are some points where it didn't predicts well.****

# **If you like my work please Upvote me..**

# **Thank You**